In [2]:
# Import dependencies:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#from imblearn.ensemble import BalancedRandomForestClassifier
#from sklearn.metrics import balanced_accuracy_score
#from imblearn.metrics import classification_report_imbalanced
#from imblearn.ensemble import EasyEnsembleClassifier

In [7]:
# Load the data:
file_path = Path("../data/Cleaned_NASA_Exoplanets.csv")
exoplanet_df = pd.read_csv(file_path)

# Drop the null columns where all values are null:
#exoplanet_df = exoplanet_df.dropna(axis='columns', how='all')

# Drop the null rows:
#exoplanet_df = exoplanet_df.dropna()

exoplanet_df.head()

C:\Users\Pritpal\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,rowid,pl_name,hostname,pl_letter,hd_name,hip_name,tic_id,gaia_id,default_flag,sy_snum,...,sy_kepmagerr2,rowupdate,pl_pubdate,releasedate,pl_nnotes,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec
0,1,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,1,2,...,NaN,5/14/2014,2008-01,5/14/2014,2,1,2,0,0,0
1,2,11 Com b,11 Com,b,HD 107383,HIP 60202,TIC 72437047,Gaia DR2 3946945413106333696,0,2,...,NaN,7/23/2014,2011-08,7/23/2014,2,1,2,0,0,0
2,3,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,1,1,...,NaN,9/4/2018,2017-03,9/6/2018,0,1,1,0,0,0
3,4,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,0,1,...,NaN,4/25/2018,2009-10,5/14/2014,0,1,1,0,0,0
4,5,11 UMi b,11 UMi,b,HD 136726,HIP 74793,TIC 230061010,Gaia DR2 1696798367260229376,0,1,...,NaN,4/25/2018,2011-08,7/23/2014,0,1,1,0,0,0


In [5]:
# Count distinct values in "soltype" column:
exoplanet_df['soltype'].value_counts()

Published Confirmed                               16151
Kepler Project Candidate (q1_q17_dr25_sup_koi)     2669
Kepler Project Candidate (q1_q16_koi)              2661
Kepler Project Candidate (q1_q17_dr25_koi)         2652
Kepler Project Candidate (q1_q17_dr24_koi)         2641
Kepler Project Candidate (q1_q12_koi)              2625
Kepler Project Candidate (q1_q8_koi)               2272
Published Candidate                                 771
TESS Project Candidate                              184
Name: soltype, dtype: int64

In [8]:
# Create a new dataframe by selecting desired columns:
new_exoplanet_df = exoplanet_df[["pl_name", "hostname", "pl_letter", "sy_snum", 
                                 "sy_pnum", "discoverymethod", "disc_instrument", 
                                 "pl_orbper", "pl_rade", "soltype", "pl_eqt", 
                                 "st_teff", "st_rad", "st_mass", "st_logg"]]


print(new_exoplanet_df.shape)
new_exoplanet_df.head()

(32626, 15)


,pl_name,hostname,pl_letter,sy_snum,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
0,11 Com b,11 Com,b,2,1,Radial Velocity,Coude Echelle Spectrograph,326.03000,NaN,Published Confirmed,NaN,4742.0,19.00,2.70,2.31
1,11 Com b,11 Com,b,2,1,Radial Velocity,Coude Echelle Spectrograph,NaN,NaN,Published Confirmed,NaN,NaN,NaN,2.60,NaN
2,11 UMi b,11 UMi,b,1,1,Radial Velocity,Coude Echelle Spectrograph,516.21997,NaN,Published Confirmed,NaN,4213.0,29.79,2.78,1.93
3,11 UMi b,11 UMi,b,1,1,Radial Velocity,Coude Echelle Spectrograph,516.22000,NaN,Published Confirmed,NaN,4340.0,24.08,1.80,1.60
4,11 UMi b,11 UMi,b,1,1,Radial Velocity,Coude Echelle Spectrograph,NaN,NaN,Published Confirmed,NaN,NaN,NaN,1.70,NaN


In [9]:
# Dropna from new dataframe:
new_exoplanet_df= new_exoplanet_df.dropna()
print(new_exoplanet_df.shape)
new_exoplanet_df.head()

(14132, 15)


,pl_name,hostname,pl_letter,sy_snum,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
100,55 Cnc e,55 Cnc,e,2,5,Radial Velocity,High Resolution Spectrograph,7.365437e-01,2.080,Published Confirmed,1958.0,5234.0,0.94,0.91,4.45
161,BD+20 594 b,BD+20 594,b,1,1,Transit,Kepler CCD Array,4.168550e+01,2.230,Published Confirmed,546.0,5766.0,0.93,0.96,4.50
190,CFHTWIR-Oph 98 b,CFHTWIR-Oph 98 A,b,1,1,Imaging,WFC3 Camera,8.040000e+06,20.849,Published Confirmed,1800.0,2320.0,0.24,0.01,3.85
193,COCONUTS-2 b,COCONUTS-2 A,b,1,1,Imaging,SpeX,4.020000e+08,12.442,Published Confirmed,434.0,3406.0,0.39,0.37,4.83
196,CoRoT-1 b,CoRoT-1,b,1,1,Transit,CoRoT CCD Array,1.508969e+00,17.385,Published Confirmed,1915.0,5950.0,1.13,0.95,4.31


In [10]:
# Drop duplicates:
new_exoplanet_df = new_exoplanet_df.drop_duplicates()
print(new_exoplanet_df.shape)
new_exoplanet_df.head()

(14131, 15)


,pl_name,hostname,pl_letter,sy_snum,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
100,55 Cnc e,55 Cnc,e,2,5,Radial Velocity,High Resolution Spectrograph,7.365437e-01,2.080,Published Confirmed,1958.0,5234.0,0.94,0.91,4.45
161,BD+20 594 b,BD+20 594,b,1,1,Transit,Kepler CCD Array,4.168550e+01,2.230,Published Confirmed,546.0,5766.0,0.93,0.96,4.50
190,CFHTWIR-Oph 98 b,CFHTWIR-Oph 98 A,b,1,1,Imaging,WFC3 Camera,8.040000e+06,20.849,Published Confirmed,1800.0,2320.0,0.24,0.01,3.85
193,COCONUTS-2 b,COCONUTS-2 A,b,1,1,Imaging,SpeX,4.020000e+08,12.442,Published Confirmed,434.0,3406.0,0.39,0.37,4.83
196,CoRoT-1 b,CoRoT-1,b,1,1,Transit,CoRoT CCD Array,1.508969e+00,17.385,Published Confirmed,1915.0,5950.0,1.13,0.95,4.31


In [11]:
# Count distinct values in "soltype" column
new_exoplanet_df['soltype'].value_counts()

Kepler Project Candidate (q1_q16_koi)         2659
Kepler Project Candidate (q1_q17_dr25_koi)    2652
Kepler Project Candidate (q1_q17_dr24_koi)    2640
Kepler Project Candidate (q1_q12_koi)         2623
Kepler Project Candidate (q1_q8_koi)          2272
Published Confirmed                           1283
Published Candidate                              2
Name: soltype, dtype: int64

In [12]:
# Drop planets with more than 1 star:
new_exoplanet_df.drop(new_exoplanet_df.index[new_exoplanet_df['sy_snum'] > 1], inplace=True)
print(new_exoplanet_df.shape)
new_exoplanet_df.head()

(13434, 15)


,pl_name,hostname,pl_letter,sy_snum,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
161,BD+20 594 b,BD+20 594,b,1,1,Transit,Kepler CCD Array,4.168550e+01,2.230,Published Confirmed,546.0,5766.0,0.93,0.96,4.50
190,CFHTWIR-Oph 98 b,CFHTWIR-Oph 98 A,b,1,1,Imaging,WFC3 Camera,8.040000e+06,20.849,Published Confirmed,1800.0,2320.0,0.24,0.01,3.85
193,COCONUTS-2 b,COCONUTS-2 A,b,1,1,Imaging,SpeX,4.020000e+08,12.442,Published Confirmed,434.0,3406.0,0.39,0.37,4.83
196,CoRoT-1 b,CoRoT-1,b,1,1,Transit,CoRoT CCD Array,1.508969e+00,17.385,Published Confirmed,1915.0,5950.0,1.13,0.95,4.31
197,CoRoT-1 b,CoRoT-1,b,1,1,Transit,CoRoT CCD Array,1.508956e+00,16.700,Published Confirmed,1898.0,5950.0,1.11,0.95,4.25


In [13]:
# Drop the "sy_snum" column as it now only has 1 unique value:
clean_exoplanet_df = new_exoplanet_df.drop(columns=['sy_snum'])
print(clean_exoplanet_df.shape)
clean_exoplanet_df.head()

(13434, 14)


,pl_name,hostname,pl_letter,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
161,BD+20 594 b,BD+20 594,b,1,Transit,Kepler CCD Array,4.168550e+01,2.230,Published Confirmed,546.0,5766.0,0.93,0.96,4.50
190,CFHTWIR-Oph 98 b,CFHTWIR-Oph 98 A,b,1,Imaging,WFC3 Camera,8.040000e+06,20.849,Published Confirmed,1800.0,2320.0,0.24,0.01,3.85
193,COCONUTS-2 b,COCONUTS-2 A,b,1,Imaging,SpeX,4.020000e+08,12.442,Published Confirmed,434.0,3406.0,0.39,0.37,4.83
196,CoRoT-1 b,CoRoT-1,b,1,Transit,CoRoT CCD Array,1.508969e+00,17.385,Published Confirmed,1915.0,5950.0,1.13,0.95,4.31
197,CoRoT-1 b,CoRoT-1,b,1,Transit,CoRoT CCD Array,1.508956e+00,16.700,Published Confirmed,1898.0,5950.0,1.11,0.95,4.25


In [16]:
# Remove the `Published Confirmed` from soltype:
confirmed_mask = clean_exoplanet_df['soltype'] != 'Published Confirmed'
clean_exoplanet_df = clean_exoplanet_df.loc[confirmed_mask]

print(clean_exoplanet_df.shape)
clean_exoplanet_df.head()

(12361, 14)


,pl_name,hostname,pl_letter,sy_pnum,discoverymethod,disc_instrument,pl_orbper,pl_rade,soltype,pl_eqt,st_teff,st_rad,st_mass,st_logg
331,EPIC 201595106 b,EPIC 201595106,b,1,Transit,Kepler CCD Array,0.877030,1.20,Published Candidate,1874.0,5711.17,0.99,1.02,4.45
428,EPIC 228836835 b,EPIC 228836835,b,1,Transit,Kepler CCD Array,0.728130,1.27,Published Candidate,932.0,3506.00,0.42,0.42,4.80
738,HAT-P-11 b,HAT-P-11,b,2,Transit,2K CCD Sensor,4.887803,4.84,Kepler Project Candidate (q1_q12_koi),801.0,4777.00,0.77,0.83,4.59
744,HAT-P-11 b,HAT-P-11,b,2,Transit,2K CCD Sensor,4.887803,4.82,Kepler Project Candidate (q1_q17_dr25_koi),800.0,4778.00,0.76,0.83,4.59
745,HAT-P-11 b,HAT-P-11,b,2,Transit,2K CCD Sensor,4.887800,4.68,Kepler Project Candidate (q1_q8_koi),794.0,4766.00,0.74,0.79,4.59


# Mockup/ Draft Model
# Split the Data into Training and Testing

### Create a Supervised Machine Learning model, use Logistic Regression // classification 1 or 0.

In [17]:
# Create our features:
#X = clean_exoplanet_df.drop("soltype", axis=1)

#X = pd.get_dummies(X)

# Create our target:
#y = clean_exoplanet_df["soltype"]

In [19]:
# X.describe() ??

In [21]:
# Check the balance of our target values
#y.value_counts()

In [22]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)